In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_csv=pd.read_csv("/kaggle/input/cassava-leaf-disease-classification/train.csv")

In [ ]:
train_csv.head()

In [ ]:
import pandas as pd
import numpy as np
train_csv
train_csv['split'] = np.random.randn(train_csv.shape[0], 1)

msk = np.random.rand(len(train_csv)) <= 0.7

train = train_csv[msk]
val = train_csv[~msk]

In [ ]:
train.drop(columns=["split",],inplace=True)
val.drop(columns=["split",],inplace=True)

In [ ]:
train.head()

In [ ]:
train.reset_index(inplace=True)
train.head()

In [ ]:
val.reset_index(inplace=True)
val.head()

In [ ]:
train.label.unique()

In [ ]:
sample_sub=pd.read_csv("/kaggle/input/cassava-leaf-disease-classification/sample_submission.csv")

In [ ]:
sample_sub

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


In [ ]:
from PIL import Image
import os
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
class CustomDataset(Dataset):
    def __init__(self, csv_file, id_col, target_col, root_dir, transform=None):
        """
        Args:
            csv_file   (string):             Path to the csv file with annotations.
            root_dir   (string):             Directory with all the images.
            id_col     (string):             csv id column name.
            target_col (string):             csv target column name.
            sufix      (string, optional):   Optional sufix for samples.
            transform  (callable, optional): Optional transform to be applied on a sample.
        """
        self.data      = csv_file
        self.id        = id_col
        self.target    = target_col
        self.root      = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.loc[idx, self.id]
        image = Image.open(os.path.join(self.root, img_name))
        if self.transform is not None:
            image = self.transform(image)
        label = self.data.loc[idx, self.target]
        return image, label

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

params = {
    'id_col':     'image_id',  
    'target_col': 'label' ,
    'transform':  data_transforms['train']
}
train_dir="../input/cassava-leaf-disease-classification/train_images"
#train_csv="../input/cassava-leaf-disease-classification/train.csv"
train_dataset = CustomDataset(csv_file=train, root_dir=train_dir, **params)
val_dataset=CustomDataset(csv_file=val, root_dir=train_dir, **params)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader=DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=0)
dataloaders={"train":train_loader,"val":val_loader}
dataset_sizes = {"train": len(train_dataset) ,"val":len(val_dataset)}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))
# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[classes])

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                dataloader=train_loader
            else:
                model.eval()   # Set model to evaluate mode
                dataloader=val_loader

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels=torch.tensor(labels)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(preds[j]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
import cv2 
  
# Save image in set directory 
# Read RGB image 
img = cv2.imread('../input/cassava-leaf-disease-classification/test_images/2216849948.jpg') 
img.size

In [ ]:
cache_dir = "/root/.cache/torch/checkpoints/"
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

In [ ]:
!cp ../input/pretrained-pytorch-models/* $cache_dir

In [ ]:
!ls $cache_dir

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 5)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

In [ ]:
visualize_model(model_ft)

In [ ]:
test=pd.read_csv("/kaggle/input/cassava-leaf-disease-classification/sample_submission.csv")

In [ ]:
test

In [ ]:
test_dir="../input/cassava-leaf-disease-classification/test_images"
#train_csv="../input/cassava-leaf-disease-classification/train.csv"
test_dataset = CustomDataset(csv_file=test, root_dir=test_dir, **params)

In [ ]:
test_loader=DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers=0)

In [ ]:
def visualize_model_for_test(model, num_images=1):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(test_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot()
                ax.axis('off')
                ax.set_title('predicted: {}'.format(preds[j]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return preds
        model.train(mode=was_training)

In [ ]:
preds=visualize_model_for_test(model_ft)

In [ ]:
preds

In [ ]:
y=torch.Tensor.cpu(preds).detach().numpy()

In [ ]:
y

In [ ]:
submit = pd.DataFrame({'image_id': test['image_id'].values, 'label': y})
submit.to_csv('submission.csv', index=False)